In [1]:
import pandas as pd
import backtrader as bt
import ta
import io
import os
import csv
from custom_resource import perform_grid_search,perform_backtest
from IPython.display import clear_output
import shutil

Loading BokehJS ...

In [2]:
def get_training(df, row):
    return df[(df['timestamp'] >= row['4_days_before_start']) & (df['timestamp'] < row['start'])]

In [3]:
def get_test(df, row):
    return df[(df['timestamp'] >= row['start']) & (df['timestamp'] < row['end'])]

In [4]:
groups = pd.read_csv('groups.csv')
groups['start'] = pd.to_datetime(groups['start'])
groups['end'] = pd.to_datetime(groups['end'])
groups['4_days_before_start'] = groups['start'] - pd.Timedelta(days=3)

In [5]:
df = pd.read_csv('features.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['rsi'] = ta.momentum.rsi(df['close'])
df['macd'] = ta.trend.macd(df['close'])
df['percentage_large_reversal'] = df['percentage_large'].rolling(3).mean() - df['percentage_large'].rolling(24).mean()

df['buy_percentage_large'] = df['buy_percentage_large'].rolling(3).mean() #18
df['percentage_large'] = df['percentage_large'].rolling(3).mean()

In [6]:
df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'percentage_large', 'percentage_large_reversal', 'buy_percentage_large', 'rsi', 'macd']]

In [7]:
class trendStrategy(bt.Strategy):
    params = dict(parameters={})

    def __init__(self):        
        self.trades = io.StringIO()
        self.trades_writer = csv.writer(self.trades)

        self.operations = io.StringIO()
        self.operations_writer = csv.writer(self.operations)

        self.portfolioValue = io.StringIO()
        self.portfolioValue_writer = csv.writer(self.portfolioValue)

        params = self.params.parameters
        self.percentage_large_par = params['percentage_large']
        self.percentage_large_reversal_par = params['percentage_large_reversal']
        
        self.loss_cap_par = params['loss_cap']
        
        self.buy_percentage_large_long_par = params['buy_percentage_large_long']
        self.buy_percentage_large_short_par = params['buy_percentage_large_short']
        
        self.macd_long_par = params['macd_long']
        self.macd_short_par = params['macd_short']
        
        self.rsi_long_par = params['rsi_long']
        self.rsi_short_par = params['rsi_short']
        
        self.mult = params['mult']
        

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
#         print("Datetime: {} Message: {}".format(dt, txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                ordertype = "BUY"
                self.log("BUY EXECUTED, Type: {}, Price: {}, Cost: {}, Comm: {}".format(order.info['name'], order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                ordertype = "SELL"
                self.log("SELL EXECUTED, Type: {}, Price: {}, Cost: {}, Comm: {}".format(order.info['name'], order.executed.price, order.executed.value, order.executed.comm))

                if order.info['name'] == 'STOP LOSS':
                    self.stops_triggered = self.stops_triggered + 1
    #                 print(self.profit_percentages)
                self.profit_percentages = []

            self.trades_writer.writerow([self.datas[0].datetime.datetime(0), ordertype, order.executed.price, order.executed.value, order.executed.comm])
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")
            self.log(order.Rejected)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS: {}, NET: {}'.format(trade.pnl, trade.pnlcomm))
        self.operations_writer.writerow([self.datas[0].datetime.datetime(0), trade.pnlcomm])

    def start(self):
        self.order = None 

    def get_logs(self):
        self.portfolioValue.seek(0)
        portfolioValueDf = pd.read_csv(self.portfolioValue, names=['Date', 'Value'])
        portfolioValueDf['Date'] = pd.to_datetime(portfolioValueDf['Date'])

        self.trades.seek(0)
        tradesDf = pd.read_csv(self.trades, names=['Date', 'Type', 'Price', 'Total Spent', 'Comission'])
        tradesDf['Date'] = pd.to_datetime(tradesDf['Date'])

        self.operations.seek(0)
        operationsDf = pd.read_csv(self.operations, names=['Date', 'Profit'])
        operationsDf['Date'] = pd.to_datetime(operationsDf['Date'])
        operationsDf = operationsDf.merge(portfolioValueDf, on='Date',how='left')
        operationsDf['original_value'] = operationsDf['Value'] - operationsDf['Profit']
        operationsDf['pct_change'] = (operationsDf.Value - operationsDf.original_value)/operationsDf.original_value * 100

        return portfolioValueDf, tradesDf, operationsDf

    def next(self):        
        self.close_price = self.datas[0].close[0]
        self.percentage_large = self.datas[0].percentage_large[0]
        self.percentage_large_reversal = self.datas[0].percentage_large_reversal[0]
        self.buy_percentage_large = self.datas[0].buy_percentage_large[0]
        self.macd = self.datas[0].macd[0]
        self.rsi = self.datas[0].rsi[0]

        self.portfolioValue_writer.writerow([self.datas[0].datetime.datetime(0), self.broker.getvalue()])

        if self.position:       
            i =  list(range(0, len(self.datas)))
            for (d,j) in zip(self.datas,i):
                if len(d) == (d.buflen()-1):
                    close = self.close(d,exectype=bt.Order.Market)

            pos = self.getposition(self.datas[0])
        
        if self.position:
            
            if pos.size < 0:
                pnl_percentage = ((pos.price-self.data.close[0])/pos.price) * 100
            elif pos.size > 0:
                pnl_percentage = ((self.data.close[0]-pos.price)/pos.price) * 100


            if pos.size < 0:                
                if pnl_percentage < self.loss_cap_par:
                    cls_ord = self.close()  
                else:
                    if self.rsi < self.rsi_short_par:
                        self.close()
            elif pos.size > 0:                    
                if pnl_percentage < self.loss_cap_par:
                    self.close()
                else:
                    if self.rsi > self.rsi_long_par:
                        self.close()
        
        if not self.position:
            if self.percentage_large_reversal > self.percentage_large_reversal_par and self.percentage_large > self.percentage_large_par:
                if self.buy_percentage_large > self.buy_percentage_large_long_par and self.macd > self.macd_long_par:
                    self.order_target_percent(target=.99)

                if self.buy_percentage_large < self.buy_percentage_large_short_par and self.macd < self.macd_short_par:
                    self.order_target_percent(target=-.99)

In [8]:
class PandasData_Custom(bt.feeds.PandasData):
    lines = ('percentage_large', 'percentage_large_reversal', 'buy_percentage_large', 'rsi', 'macd', )
    params = (
        ('datetime', 0),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 5),
        ('percentage_large', 6),
        ('percentage_large_reversal', 7),
        ('buy_percentage_large', 8),
        ('rsi', 9),
        ('macd', 10)
    )

In [9]:
all_params = {
    'percentage_large': [0.12, 0.18, 0.24, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7],
    'percentage_large_reversal': [0, 0.001, 0.005, 0.01, 0.02, 0.03, 0.06, 0.12, 0.24, 0.5],
    'loss_cap': [-0.5,-1,-2,-3],
    'buy_percentage_large_long': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'buy_percentage_large_short': [0,0.01,0.05,0.1,0.15,0.2,0.3,0.5],
    'mult': [1],
    'macd_long': [-80,-60,-40, -30, -20, 0, 10],
    'macd_short': [-60, -50, -40, -30, -20, -10, 0],
    'rsi_long': [75,80,85,90],
    'rsi_short': [5,10,15,20,25,30,35]
}

In [10]:
# if os.path.isdir("bokeh/"):
#     shutil.rmtree("bokeh/")

all_res = pd.DataFrame()

for idx, row in groups.iterrows():
    print(row)
    curr_res = {}

    training_df = get_training(df, row)
    test_df = get_test(df, row)

    curr_res['group_name'] = row['name']
    
#     if row['name'] == 'corona_recovery':
#         iterations = 50
#     else:
#         iterations = 850
        
    iterations = 850

    training_grid = perform_grid_search(training_df, all_params, trendStrategy, PandasData_Custom, iterations=iterations)

    best_ones = training_grid.iloc[0][all_params.keys()].to_dict()

    for idx, val in best_ones.items():
        if idx != 'mult':
            curr_res[idx + "_par"] = val

    test_result = perform_backtest(best_ones, test_df, trendStrategy, PandasData_Custom)

    curr_res['return_%'] = test_result['Return (%)']
    curr_res['sharpe'] = test_result['sharpe']
    
    curr_res['bokeh'] = perform_backtest(best_ones, test_df, trendStrategy, PandasData_Custom, bokeh=True)
    clear_output(wait=True)
    
    best_ones['mult'] = 3
    test_result = perform_backtest(best_ones, test_df, trendStrategy, PandasData_Custom)
    curr_res['lev_3_ret'] = test_result['Return (%)']

    best_ones['mult'] = 5
    test_result = perform_backtest(best_ones, test_df, trendStrategy, PandasData_Custom)
    curr_res['lev_5_ret'] = test_result['Return (%)']

    best_ones['mult'] = 10
    test_result = perform_backtest(best_ones, test_df, trendStrategy, PandasData_Custom)
    curr_res['lev_10_ret'] = test_result['Return (%)']

    curr_res['hodl_return'] = round((test_df.iloc[-1]['close'] - test_df.iloc[0]['close'])/test_df.iloc[0]['close'] * 100, 2)
    all_res = all_res.append(curr_res, ignore_index=True)
    print(all_res)

          bokeh  buy_percentage_large_long_par  \
0   73222c.html                            0.9   
1   93b48a.html                            0.3   
2   f20cf1.html                            0.4   
3   73dd38.html                            0.2   
4   7fa641.html                            0.2   
5   e2189f.html                            0.3   
6   1903d0.html                            0.4   
7   8777e5.html                            0.6   
8   345063.html                            0.8   
9   4b35f3.html                            0.5   
10  a6c740.html                            0.7   

    buy_percentage_large_short_par               group_name  hodl_return  \
0                             0.50              corona_dump       -33.87   
1                             0.50          corona_recovery        27.68   
2                             0.01         further_recovery        26.26   
3                             0.50          9k_accumulation        -1.07   
4                  

In [12]:
all_res

,bokeh,buy_percentage_large_long_par,buy_percentage_large_short_par,group_name,hodl_return,lev_10_ret,lev_3_ret,lev_5_ret,loss_cap_par,macd_long_par,macd_short_par,percentage_large_par,percentage_large_reversal_par,return_%,rsi_long_par,rsi_short_par,sharpe
0,73222c.html,0.9,0.50,corona_dump,-33.87,-17.84,89.15,106.50,-3.0,-80.0,0.0,0.12,0.001,32.90,90.0,25.0,2.948778
1,93b48a.html,0.3,0.50,corona_recovery,27.68,-77.29,-12.14,-31.89,-1.0,-20.0,-10.0,0.45,0.000,-0.48,85.0,15.0,0.264799
2,f20cf1.html,0.4,0.01,further_recovery,26.26,260.98,78.30,130.49,-0.5,-60.0,0.0,0.12,0.001,26.10,90.0,35.0,5.086729
3,73dd38.html,0.2,0.50,9k_accumulation,-1.07,-93.39,-46.25,-67.15,-2.0,-30.0,-40.0,0.12,0.060,-17.48,85.0,10.0,-3.450241
4,7fa641.html,0.2,0.10,further_9k_accumulation,-6.02,-55.31,-17.88,-29.41,-2.0,-60.0,-30.0,0.60,0.120,-6.00,75.0,15.0,-4.696855
5,e2189f.html,0.3,0.00,final_9k_accumulation,5.40,34.87,9.84,16.69,-2.0,-60.0,-50.0,0.12,0.240,3.22,80.0,25.0,4.902501
6,1903d0.html,0.4,0.15,initial_takeoff,22.87,274.48,73.82,128.09,-0.5,-20.0,-40.0,0.35,0.030,23.43,90.0,30.0,4.990975
7,8777e5.html,0.6,0.50,12k_dump,-3.70,-63.04,-23.93,-37.30,-2.0,-20.0,-50.0,0.60,0.001,-8.52,90.0,30.0,-2.918238
8,345063.html,0.8,0.30,10k_accumulation,-5.43,46.13,13.84,23.07,-1.0,-30.0,-10.0,0.30,0.005,4.61,80.0,5.0,1.981538
9,4b35f3.html,0.5,0.50,10k_accumulation_2,5.13,63.35,22.95,36.86,-1.0,-60.0,-40.0,0.70,0.120,7.84,85.0,10.0,3.568781


In [13]:
all_res.to_csv('all_res.csv', index=None)

In [14]:
#also try removing percentage and just x

In [2]:
all_res = pd.read_csv('all_res.csv')
res = pd.DataFrame()

for idx, row in all_res.iterrows():
    parameters = ""
    for key in all_params.keys():
        if key != "mult":
            key = key + "_par"
        
            parameters = parameters + key + ": " + str(row[key]) + "\n"
    
    row['parameters'] = parameters
    res = res.append(row, ignore_index=True)
    
all_res = res
all_res['bokeh'] = "[Backtest](static/" + all_res['bokeh']  + ")"

NameError: name 'all_params' is not defined

In [35]:
all_res = all_res[['group_name', 'return_%', 'hodl_return', 'lev_3_ret', 'lev_5_ret', 'sharpe', 'parameters', 'bokeh']]

In [36]:
all_res

,group_name,return_%,hodl_return,lev_3_ret,lev_5_ret,sharpe,parameters,bokeh
0,corona_dump,32.90,-33.87,89.15,106.50,2.95,percentage_large_par: 0.12\npercentage_large_r...,[Backtest](static/73222c.html)
1,corona_recovery,-0.48,27.68,-12.14,-31.89,0.26,percentage_large_par: 0.45\npercentage_large_r...,[Backtest](static/93b48a.html)
2,further_recovery,26.10,26.26,78.30,130.49,5.09,percentage_large_par: 0.12\npercentage_large_r...,[Backtest](static/f20cf1.html)
3,9k_accumulation,-17.48,-1.07,-46.25,-67.15,-3.45,percentage_large_par: 0.12\npercentage_large_r...,[Backtest](static/73dd38.html)
4,further_9k_accumulation,-6.00,-6.02,-17.88,-29.41,-4.70,percentage_large_par: 0.6\npercentage_large_re...,[Backtest](static/7fa641.html)
5,final_9k_accumulation,3.22,5.40,9.84,16.69,4.90,percentage_large_par: 0.12\npercentage_large_r...,[Backtest](static/e2189f.html)
6,initial_takeoff,23.43,22.87,73.82,128.09,4.99,percentage_large_par: 0.35\npercentage_large_r...,[Backtest](static/1903d0.html)
7,12k_dump,-8.52,-3.70,-23.93,-37.30,-2.92,percentage_large_par: 0.6\npercentage_large_re...,[Backtest](static/8777e5.html)
8,10k_accumulation,4.61,-5.43,13.84,23.07,1.98,percentage_large_par: 0.3\npercentage_large_re...,[Backtest](static/345063.html)
9,10k_accumulation_2,7.84,5.13,22.95,36.86,3.57,percentage_large_par: 0.7\npercentage_large_re...,[Backtest](static/4b35f3.html)


In [37]:
all_res = all_res.round(2)

In [38]:
all_res.to_csv("selected.csv", index=None)

In [39]:
starting = 1000
vals = 1+all_res['return_%'].values/100

for x in vals:
    print(starting)
    starting = starting * x
    
print(starting)

1000
1329.0
1322.6208
1667.8248288
1376.2890487257598
1293.711705802214
1335.3692227290453
1648.2462316144606
1507.8156526809087
1577.3259542694987
1700.9883090842275
2550.972167133616


In [40]:
starting = 1000
vals = 1+all_res['hodl_return'].values/100

for x in vals:
    print(starting)
    starting = starting * x
    
print(starting)

1000
661.3
844.3478399999999
1066.0735827839999
1054.666595448211
991.1756664022287
1044.699152387949
1283.6218485390727
1236.127840143127
1169.0060984233553
1228.9761112724734
1865.4628393004875


In [41]:
starting = 1000
vals = 1+all_res['lev_3_ret'].values/100

for x in vals:
    print(starting)
    starting = starting * x
    
print(starting)

1000
1891.5000000000002
1661.8719000000003
2963.1175977000003
1592.67570876375
1307.9052920367915
1436.6031727732118
2497.1036349143965
1899.5467350793815
2162.444003214368
2658.7249019520655
6644.685274958602


In [42]:
starting = 1000
vals = 1+all_res['lev_5_ret'].values/100

for x in vals:
    print(starting)
    starting = starting * x
    
print(starting)

1000
2065.0
1406.4715
3241.7761603500003
1064.9234686749749
751.7294765376647
877.1931261718009
2000.7898014852608
1254.4952055312585
1543.90724944732
2112.9914615936023
7392.723226677536
